# Pytorch Quantization

## Setup PyTorch

In [1]:
%pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.quantization
import pathlib

## Dynamic Quantization


### Load mnist dataset 

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

train_dataset = datasets.MNIST('./data', train=True, download=True,transform=transform)
test_dataset = datasets.MNIST('./data', train=False,transform=transform)

### Train model

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(12 * 13 * 13, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)  
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  
        x = self.fc(x)
        output = F.log_softmax(x, dim=1)
        return output


train_loader = torch.utils.data.DataLoader(train_dataset, 32)
test_loader = torch.utils.data.DataLoader(test_dataset, 32)

device = "cpu"

epochs = 1

model = Net().to(device)
optimizer = optim.Adam(model.parameters())

model.train()

for epoch in range(1, epochs+1):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.346256
Train Epoch: 1 [32/60000 (0%)]	Loss: 2.331087
Train Epoch: 1 [64/60000 (0%)]	Loss: 2.174083
Train Epoch: 1 [96/60000 (0%)]	Loss: 1.948799
Train Epoch: 1 [128/60000 (0%)]	Loss: 2.063534
Train Epoch: 1 [160/60000 (0%)]	Loss: 2.173075
Train Epoch: 1 [192/60000 (0%)]	Loss: 1.654042
Train Epoch: 1 [224/60000 (0%)]	Loss: 1.806324
Train Epoch: 1 [256/60000 (0%)]	Loss: 1.894512
Train Epoch: 1 [288/60000 (0%)]	Loss: 1.542542
Train Epoch: 1 [320/60000 (1%)]	Loss: 1.602335
Train Epoch: 1 [352/60000 (1%)]	Loss: 1.444259
Train Epoch: 1 [384/60000 (1%)]	Loss: 1.384955
Train Epoch: 1 [416/60000 (1%)]	Loss: 1.297989
Train Epoch: 1 [448/60000 (1%)]	Loss: 1.231365
Train Epoch: 1 [480/60000 (1%)]	Loss: 1.590174
Train Epoch: 1 [512/60000 (1%)]	Loss: 1.377761
Train Epoch: 1 [544/60000 (1%)]	Loss: 1.096890
Train Epoch: 1 [576/60000 (1%)]	Loss: 1.383488
Train Epoch: 1 [608/60000 (1%)]	Loss: 1.371453
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.349930
Train Epoch: 1 [67

### Quantize model

In [5]:
quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

### Check model size

In [6]:
models_dir = pathlib.Path("./models/")
models_dir.mkdir(exist_ok=True, parents=True)
torch.save(model.state_dict(), "./models/original_model.p")
torch.save(quantized_model.state_dict(), "./models/quantized_model.p")

%ls -lh models

total 216K
-rw-rw-r-- 1 harzad harzad 82K Dis  20 07:29 mnist_model.onnx
-rw-rw-r-- 1 harzad harzad 82K Dis  23 10:26 original_model.p
-rw-rw-r-- 1 harzad harzad 24K Dis  20 23:25 post_quantized_model.p
-rw-rw-r-- 1 harzad harzad 23K Dis  23 10:26 quantized_model.p


### Check accuracy

In [19]:
def test(model, device, test_loader):
    model.to(device)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    return 100. * correct / len(test_loader.dataset)

original_acc = test(model, device, test_loader)
quantized_acc = test(quantized_model, device, test_loader)

print('Original model accuracy: {:.0f}%'.format(original_acc))
print('Quantized model accuracy: {:.0f}%'.format(quantized_acc))

Original model accuracy: 97%
Quantized model accuracy: 97%


In [8]:
def evaluate(model, criterion, data_loader, neval_batches):
    model.eval()
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    cnt = 0
    with torch.no_grad():
        for image, target in data_loader:
            output = model(image)
            loss = criterion(output, target)
            cnt += 1
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            print('.', end = '')
            top1.update(acc1[0], image.size(0))
            top5.update(acc5[0], image.size(0))
            if cnt >= neval_batches:
                 return top1, top5

    return top1, top5

## Post-training static quantization

In [20]:
loaded_model = Net()
loaded_model.load_state_dict(torch.load("./models/original_model.p"))
loaded_model.to("cpu")
loaded_model.eval()

loaded_model.qconfig = torch.ao.quantization.get_default_qconfig('x86')

# model_fp32_fused = torch.ao.quantization.fuse_modules(loaded_model, [['conv1']])
model_fp32_prepared = torch.ao.quantization.prepare(loaded_model)


input_fp32 = next(iter(test_loader))[0][0:1]
input_fp32.to("cpu")
model_fp32_prepared(input_fp32)
model_int8 = torch.ao.quantization.convert(model_fp32_prepared)

torch.save(model_int8.state_dict(), "./models/post_quantized_model.p")

%ls -lh models

total 220K
-rw-rw-r-- 1 harzad harzad 82K Dis  20 07:29 mnist_model.onnx
-rw-rw-r-- 1 harzad harzad 82K Dis  23 10:26 original_model.p
-rw-rw-r-- 1 harzad harzad 25K Dis  23 10:37 post_quantized_model.p
-rw-rw-r-- 1 harzad harzad 23K Dis  23 10:26 quantized_model.p


In [21]:

quantized_acc = test(model_int8, "cpu", test_loader)
print('Post quantized model accuracy: {:.0f}%'.format(quantized_acc))

NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

QuantizedCPU: registered at ../aten/src/ATen/native/quantized/cpu/qconv.cpp:1874 [kernel]
Quan